<a href="https://colab.research.google.com/github/edkiriyama/gemini_curso/blob/main/Sistema_para_busca_em_documentos_usando_embeddings_e_a_API_GEMINI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q -U google-generativeai

#Guia de Embeddings

[API GEMINI EMBEDDINGS](https://ai.google.dev/gemini-api/docs/embeddings?hl=pt-br)

In [3]:
# Import Libs

#Import the Python SDK
import google.generativeai as genai
#Used to securely store your API Key
from google.colab import userdata

#Libs para base de dados
import pandas as pd
import numpy as np


In [9]:
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [10]:
for i in genai.list_models():
  if 'embedContent'in i.supported_generation_methods:
    print(i.name)

models/embedding-001
models/text-embedding-004


[API GOOGLE GEMINI - Pesquisa por documentos](https://ai.google.dev/gemini-api/tutorials/document_search?hl=pt-br)

In [13]:
# Exemplo de embedding
title = 'A próxima geração de IA para desenvolvedores e Google Workspace'
sample_text = 'Título: A próxima geração de IA para desenvolvedores e Google Workspace'
'\n'
'Artigo completo:\n'
'Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa'

embeddings = genai.embed_content(model='models/embedding-001',
    content = sample_text,
    title = title,
    task_type='RETRIEVAL_DOCUMENT'
  )

print(embeddings)

{'embedding': [0.05492989, 0.009307224, -0.03363807, -0.023828218, 0.07841232, 0.018733714, -0.022420118, -0.0016427166, 0.03924099, 0.057140194, 0.005172891, 0.03825938, -0.036689468, 0.0022303318, 0.021313649, -0.0024103776, 0.025673402, -0.03328592, -0.020979624, -0.005710304, 0.021080395, 0.0046167844, -0.045135174, -0.049866844, -0.022381082, 0.016066447, -0.014832268, -0.04362006, 0.0035906127, 0.010002601, -0.04290806, 0.039568324, -0.019747017, 0.021192143, -0.04545719, 0.006561123, -0.046718944, -0.017930955, -0.0021596432, -0.0023262927, -0.024405124, -0.070571005, -0.019826962, 0.051137034, 0.022113815, -0.032279383, 0.032599177, 0.010793851, 0.004252053, -0.05039507, 0.0075884727, 0.018511007, 0.07136252, -0.01815533, 0.00043964433, 0.0016312439, -0.0060594906, -0.028882924, 0.015078544, -0.013932823, 0.0013604995, 0.014152038, -0.014672936, 0.04525626, 0.009853937, -0.046105225, -0.0329797, -0.0041908557, -0.004886637, 0.07067624, -0.0005316908, -0.014378384, 0.040319514, 

In [14]:
# Listagem de documentos que serão buscados
DOCUMENT1 = {
    'Título': 'A próxima geração de IA para desenvolvedores e Google Workspace',
    'Conteúdo': 'Título: A próxima geração de IA para desenvolvedores e Google Workspace'
}

DOCUMENT2 = {
    'Título': 'Artigo completo',
    'Conteúdo': 'Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa'
}

DOCUMENT3 = {
    'Título': 'Mudança de marchas',
    'Conteúdo': 'Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.'
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]



In [26]:
df = pd.DataFrame(documents)
df.columns = ['Titulo', 'Conteudo']
df

,Titulo,Conteudo
0,A próxima geração de IA para desenvolvedores e...,Título: A próxima geração de IA para desenvolv...
1,Artigo completo,Gemini API & Google AI Studio: Uma maneira ace...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [33]:
model = 'models/embedding-001'

In [34]:
def embed_fn(title, text):
  embeddings = genai.embed_content(model=model,
    content = text,
    title = title,
    task_type='RETRIEVAL_DOCUMENT')['embedding']
  return embeddings

In [35]:
df['Embedding'] = df.apply(lambda x: embed_fn(x['Titulo'], x['Conteudo']), axis=1)

In [36]:
df

,Titulo,Conteudo,Embedding
0,A próxima geração de IA para desenvolvedores e...,Título: A próxima geração de IA para desenvolv...,"[0.05492989, 0.009307224, -0.03363807, -0.0238..."
1,Artigo completo,Gemini API & Google AI Studio: Uma maneira ace...,"[0.04951636, -0.057676196, -0.016170539, 0.037..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[0.00018316552, -0.03903683, 0.0017067897, 0.0..."


In [39]:
def gerar_buscar_consulta(consulta, base, model):
  embeddings_consulta = genai.embed_content(model=model,
    content = consulta,
    task_type='RETRIEVAL_QUERY')

  produtos_escalares = np.dot(np.stack(df['Embedding']), embeddings_consulta['embedding'])
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]['Conteudo']

In [41]:
consulta = 'Como se troca a marcha do carro?'

trecho = gerar_buscar_consulta(consulta, df, model)

print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.


Usar o proprio Gemini para reescrever o resultado.

In [47]:
generation_config = {
    'candidate_count':1, # Retorna uma única resposta
    'temperature':0.5
    }

In [48]:
prompt = f'Reescreve esse texto de uma forma mais descontraída, sem adicionar informação que não façam parte do texto: {consulta}'

model_2 = genai.GenerativeModel('models/gemini-1.0-pro', generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Como dar um rolê maneiro com seu carro:**

1. **Pisa fundo:** Aperte o pedal da embreagem até o final.
2. **Mexa a alavanca:** Mova a alavanca de câmbio para a marcha que você quer.
3. **Solta a embreagem:** Vá soltando o pedal da embreagem devagar enquanto dá uma aceleradinha.
4. **Curte o passeio:** Aproveita o rolê com seu carro maneiro!
